In [2]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
#from langchain.vectorstores import Chroma
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

/home/patriciapmr/.pyenv/versions/3.10.6/envs/chatbot_movies/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [4]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [5]:
genai.configure(api_key=GOOGLE_API_KEY)

In [102]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [85]:
result = llm.invoke("Write a ballad about LangChain")
print(result.content)

In realms of code, a tale unfurls,
Of LangChain, a marvel that whirls.
A bard of language, a master of prose,
Its prowess in translation, it shows.

From ancient tongues to languages new,
LangChain weaves words, both old and true.
With algorithms, it decodes the unknown,
Bridging cultures, making knowledge known.

In classrooms and boardrooms, it's a guide,
Translating thoughts with grace and stride.
Students learn, businesses thrive,
As LangChain's magic helps them survive.

It whispers secrets in foreign ears,
Breaking down barriers, quelling fears.
Travelers find solace in its embrace,
As it guides them through unfamiliar space.

Like a minstrel's lyre, it plucks at strings,
Creating melodies that language brings.
In every verse, a story unfolds,
As LangChain's power, the world beholds.

So let us sing of LangChain's might,
A beacon of knowledge, a shining light.
May its translation magic ever flow,
Connecting hearts and minds, as it grows.


In [7]:
chroma = Chroma(persist_directory="../raw_data/chroma_db", embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"))

In [9]:
print(chroma)

In [10]:
chroma.get()['documents']

['id: 1003698\nTitle: The Idiots\nYear: 1998.0\nDescription: With his first Dogma-95 film director Lars von Trier opens up a completely new film platform. With a mix of home-video and documentary styles the film tells the story of a group of young people who have decided to get to know their “inner-idiots” and thus not only facing and breaking their outer appearance but also their inner.\nMinutes: 114.0\nRating: 3.51\nGenre: Comedy, Drama\nActors: Ewald Larsen, Christian Friis, Louise B. Clausen, Lone Lindorff, Erno Müller, Regitze Estrup, Lotte Munk, Marina Bouras, Lars von Trier, Birgit Conradi, Julie Wieth, Albert Wickmann, Ditlev Weddelsborg, Jesper Sønderaas, Peter Frøge, Lars Bjarke, Kirsten Vaupel, John Martinus, Lillian Tillegreen, Claus Strandberg, Jens Jørn Spottag, Bodil Jørgensen, Jens Albinus, Anne Louise Hassing, Troels Lyby, Nikolaj Lie Kaas, Louise Mieritz, Henrik Prip, Luis Mesonero, Hans Henrik Clemensen, Trine Michelsen, Anders Hove, Michael Moritzen, Jan Elle, Knud 

In [11]:
retriever = chroma.as_retriever()

In [12]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7fca1265ea70>)

In [16]:
query = "Kate Winslet"
docs = chroma.similarity_search(query)
docs

[Document(page_content="id: 1053017\nTitle: Purple Violets\nYear: 2007.0\nDescription: Patti Petalson is a promising writer, but her marriage and conventional job keep her from her dream. She longs to return to her writing, especially after running into her first love Brian Callahan, a successful crime novelist. Kate is Patti's best friend since college; she's a tough-talking schoolteacher who plays therapist to all Patti's problems, while she's got a few of her own.\nMinutes: 103.0\nRating: 3.09\nGenre: Romance, Comedy, Drama\nActors: Peter Van Wagner, Bill Hader, Joe Pistone, Peter Jacobson, Jay Patterson, Donal Logue, Bob Wiltfong, PT Walkley, Rosemarie DeWitt, Sarah Hudnut, Heather Goldenhersh, Dara Coleman, Dennis Farina, Debra Messing, Edward Burns, Patrick Wilson, Selma Blair, Elizabeth Reaser\nTheme: Catchy songs and hilarious musical comedy, Dazzling vocal performances and musicals, Laugh-out-loud relationship entanglements, Captivating relationships and charming romance, Song

In [17]:
template = """
You are a helpful AI assistant.
Answer based on the context provided.
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
#Invoke the retrieval chain
response=retrieval_chain.invoke({"input":"Tell me about Kate Winslet?"})
#Print the answer to the question
print(response["answer"])

The provided context does not mention Kate Winslet.


In [45]:
#